In [34]:
import pandas as pd
import os

# Read the variable names from the Excel file
variable_df_y = pd.read_excel("../../scripting/boundary_conditions/wlwq_allsamples/TFV_mapping_keys.xlsx", sheet_name="Model_TFV_Y")
required_variable = variable_df_y["Variable Name"].str.replace(' ', '', regex=False).values

# Create a mapping from Variable Name to TFV Variable Name
name_mapping = {row["Variable Name"].replace(' ', ''): row["TFV Variable Name"] 
                 for _, row in variable_df_y.iterrows()}

# Initialize list for initial values
init_val_lst = []
target_date = pd.Timestamp('2023-01-01 00:00:00')

# Define the directory containing the CSV files
data_dir = "../../data-warehouse/csv/ht/wlwq/"

# Loop through all CSV files in the directory
for filename in os.listdir(data_dir):
    if filename.endswith("_profile_Data.csv"):  # Check for profile Data files
        # Check if any required_variable is in the filename
        if any(var in filename for var in required_variable):
            variable_data = os.path.join(data_dir, filename)
            df = pd.read_csv(variable_data, encoding='unicode_escape')

            # Convert 'Date' column to datetime
            df['Date'] = pd.to_datetime(df['Date'])

            # Check if 2023 data is available
            if df['Date'].dt.year.isin([2023]).any():
                # Get values around the target date
                if target_date in df['Date'].values:
                    init_val = df.loc[df['Date'] == target_date, 'Data'].values[0]
                else:
                    idx = df['Date'].searchsorted(target_date)
                    before = df.iloc[idx - 1]
                    after = df.iloc[idx]

                    # Calculate the median
                    init_val = (before['Data'] + after['Data']) / 2
            else:
                init_val = "no available data from 2023"

            var = os.path.basename(variable_data).replace('_profile_Data', '').split('.')[0]
            init_val_lst.append([var, init_val])

# Create a DataFrame from the list
result_df = pd.DataFrame(init_val_lst, columns=['Variable', 'Initial Value'])

# Extract the word after the underscore for matching
result_df['MatchWord'] = result_df['Variable'].str.split('_').str[-1]

# Add TFV Variable Name from the mapping
result_df['TFV Variable Name'] = result_df['MatchWord'].map(name_mapping)

# Drop rows where TFV Variable Name is NaN
result_df = result_df.dropna(subset=['TFV Variable Name'])

# Drop the MatchWord column as it's no longer needed
result_df = result_df.drop(columns=['MatchWord'])

# # Sort by the word after the underscore in the "Variable" column
# result_df['SortKey'] = result_df['Variable'].str.split('_').str[-1]
# result_df = result_df.sort_values(by=['SortKey']).drop(columns=['SortKey'])
result_df = result_df.sort_values(by=['TFV Variable Name'])

# Save to a CSV file
result_df.to_csv('initial_values.csv', index=False)

print(result_df)


                                             Variable  \
40                     WoodsLakeMiddle_AirTemperature   
41  WoodsLakeMiddleLakeBedContinuous_SpecificCondu...   
28               WoodsLakeMiddle_SpecificConductivity   
53          WoodsLakeMiddleLakeBedContinuous_Salinity   
14                           WoodsLakeMiddle_Salinity   
2                         WoodsLakeMiddle_Temperature   
12       WoodsLakeMiddleLakeBedContinuous_Temperature   
21                     WoodsLakeMiddle_MeanWaveHeight   
43                WoodsLakeMiddleLakeBedContinuous_pH   
22                                 WoodsLakeMiddle_pH   
19                    WoodsLakeMiddle_TotalAlkalinity   
18     WoodsLakeMiddle_TotalOxidisedNitriteandNitrate   
39    WoodsLakeMiddle_TotalOrganicCarbonNon-Purgeable   
37      WoodsLakeMiddleLakeBedContinuous_O2Saturation   
20                       WoodsLakeMiddle_O2Saturation   
52            WoodsLakeMiddle_TotalReactivePhosphorus   
56                      WoodsLa